In [0]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = spark.sql("SELECT * FROM classes.gold.df_atendimentos").toPandas()

df['data_atendimento'] = pd.to_datetime(df['data_atendimento'])
df['ano'] = df['data_atendimento'].dt.year

top_localidades = df['localidade'].value_counts().head(20).reset_index()
top_localidades.columns = ['localidade', 'total_atendimentos']

# MÉDIA DE ATRASO POR LOCALIDADE (diferença_atendimento_consulta)
atraso_por_local = df.groupby('localidade')['diferença_atendimento_consulta'].agg(['mean', 'count'])
atraso_por_local = atraso_por_local[atraso_por_local['count'] >= 10]  # filtra locais com poucos dados
atraso_por_local = atraso_por_local.sort_values('mean', ascending=False).head(15)

# DASHBOARD 3 EM 1 — PROFISSIONAL E PRONTO PARA BANCA
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Top 20 Localidades" 
        "por Volume de Atendimentos",
        "Média de Atraso (em meses) por Localidade",
        "Distribuição de Atrasos (Todos os dados)",
        "Evolução Anual por Top 5 Localidades"
    ),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "histogram"}, {"type": "xy"}]],
    vertical_spacing=0.20,
    horizontal_spacing=0.30
)

# 1 — Top 20 localidades (barras horizontais)
fig.add_trace(
    go.Bar(y=top_localidades['localidade'], x=top_localidades['total_atendimentos'],
           orientation='h', marker_color='#1f77b4', name='Atendimentos'),
    row=1, col=1
)

# 2 — Atraso médio por localidade (vermelho = atraso alto)
fig.add_trace(
    go.Bar(x=atraso_por_local.index, y=atraso_por_local['mean'].round(1),
           marker_color='crimson', name='Atraso Médio (meses)'),
    row=1, col=2
)

# 3 — Histograma de todos os atrasos
fig.add_trace(
    go.Histogram(x=df['diferença_atendimento_consulta'], nbinsx=50,
                 marker_color='lightblue', name='Distribuição de Atrasos'),
    row=2, col=1
)

# 4 — Evolução anual das top 5 localidades
top5_locais = df['localidade'].value_counts().head(5).index
evolucao = df[df['localidade'].isin(top5_locais)].groupby(['ano', 'localidade']).size().reset_index(name='count')

for loc in top5_locais:
    data_loc = evolucao[evolucao['localidade'] == loc]
    fig.add_trace(
        go.Scatter(x=data_loc['ano'], y=data_loc['count'], mode='lines+markers',
                   name=loc, line=dict(width=3)),
        row=2, col=2
    )

# ESTILO FINAL — FICA INCRÍVEL
fig.update_layout(
    height=900,
    title_text="<b>Análise Completa de Atendimentos por Localidade (2020–2025)</b>",
    title_x=0.5,
    showlegend=False,
    font=dict(size=12.5)
)

fig.update_xaxes(title_text="Localidade", row=1, col=1)
fig.update_yaxes(title_text="Total de Atendimentos", row=1, col=1)
fig.update_xaxes(title_text="Localidade", tickangle=45, row=1, col=2)
fig.update_yaxes(title_text="Atraso Médio (meses)", row=1, col=2)
fig.update_xaxes(title_text="Diferença (meses)", row=2, col=1)
fig.update_yaxes(title_text="Frequência", row=2, col=1)
fig.update_xaxes(title_text="Ano", tickmode='linear', row=2, col=2)
fig.update_yaxes(title_text="Atendimentos", row=2, col=2)

fig.show()